### Merging NED13 tiles and generating hillshaded color-relief terrain maps

In [ ]:
import os
import sys
import json
import glob
import datetime
import rasterio
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

sys.path.insert(0, '../')
from managers import managers

%load_ext autoreload
%autoreload 2

In [ ]:
# load the raw NED13 tile directories
ned13_dirs = [p for p in glob.glob('/media/keith/USGS_Backup/USGS/NED13/*') if os.path.isdir(p)]
ned13_dirs.sort()

In [ ]:
sys.path.append('../../../_projects-db/sierra-map/src/')
import gdal_wrappers as gw

In [ ]:
# downsample and reproject tiles as tifs
# (useful for quickly testing various ROIs)
for path in ned13_dirs:
    d = managers.datasets.new_dataset('ned13', path, exists=True)
    output_filename = '/media/keith/USGS_Backup/USGS/NED13-100m/%s_100m.tif' % d.name
    if os.path.isfile(output_filename):
        continue

    print(d.name)
    managers.utils.shell(
        gw.warp(d.bandpath(), output_filename, r='bilinear', tr=(100, 100), t_srs='EPSG:3857'))

In [ ]:
# all of the downsampled tiles
dataset_paths = glob.glob('/media/keith/USGS_Backup/USGS/NED13-100m/*.tif')

In [ ]:
# manually-defined ROIs
bounds = {
    'guerneville': [-123.8, 38.4, -122.8, 39.],
    'south-bay': [-122.55, 36.93, -121.75, 37.65],
    'berkeley': [-122.38, 37.75, -122.06, 38.0],
    'sf': [-122.54, 37.64, -122.34, 37.81],
    'moab': [-109.92, 38.25, -109.16, 38.95],
    'moab-wide': [-110.5, 38, -109, 39.5],
    'markleeville': [-120.194, 38.269, -119.361, 39]
}

In [ ]:
# original sierra map cmap
cmap_orig = [
    {'elevation': 0.0, 'color': (25, 125, 225)},
    {'elevation': 3.28, 'color': (110, 140, 100)},
    {'elevation': 328.0, 'color': (115, 150, 105)},
    {'elevation': 1312.0, 'color': (140, 155, 115)},
    {'elevation': 1640.0, 'color': (190, 204, 145)},
    {'elevation': 3280.0, 'color': (250, 250, 185)},
    {'elevation': 4920.0, 'color': (250, 205, 160)},
    {'elevation': 6560.0, 'color': (230, 180, 155)},
    {'elevation': 8200.0, 'color': (240, 200, 190)},
    {'elevation': 9840.0, 'color': (245, 220, 210)},
    {'elevation': 11480.0, 'color': (253, 235, 230)},
    {'elevation': 13120.0, 'color': (255, 255, 255)}]

In [ ]:
# original sierra map adapted to norcal (4800ft - 10000ft)
cmap_markleeville = [
    {'elevation': 0, 'color': (25, 125, 225)},
    {'elevation': 5000, 'color': (110, 140, 100)},
    {'elevation': 5500, 'color': (115, 150, 105)},
    {'elevation': 6000, 'color': (140, 155, 115)},
    {'elevation': 6500, 'color': (190, 204, 145)},
    {'elevation': 7000, 'color': (250, 250, 185)},
    {'elevation': 7500, 'color': (250, 205, 160)},
    {'elevation': 8000, 'color': (230, 180, 155)},
    {'elevation': 8500, 'color': (240, 200, 190)},
    {'elevation': 9000, 'color': (245, 220, 210)},
    {'elevation': 9500, 'color': (253, 235, 230)},
    {'elevation': 10000, 'color': (255, 255, 255)}]

In [ ]:
# original sierra map adapted to norcal (4800ft - 10000ft)
cmap_markleeville_banded = [
    {'elevation': 0, 'color': (25, 125, 225)},
    {'elevation': 5000, 'color': (110, 140, 100)},
    {'elevation': 5500, 'color': (110, 140, 100)},
    {'elevation': 5501, 'color': (115, 150, 105)},
    {'elevation': 6000, 'color': (115, 150, 105)},
    {'elevation': 6001, 'color': (140, 155, 115)},
    {'elevation': 6500, 'color': (140, 155, 115)},
    {'elevation': 6501, 'color': (190, 204, 145)},
    {'elevation': 7000, 'color': (190, 204, 145)},
    {'elevation': 7001, 'color': (250, 250, 185)},
    {'elevation': 7500, 'color': (250, 250, 185)},
    {'elevation': 7501, 'color': (250, 205, 160)},
    {'elevation': 8000, 'color': (250, 205, 160)},
    {'elevation': 8001, 'color': (230, 180, 155)},
    {'elevation': 8500, 'color': (230, 180, 155)},
    {'elevation': 8501, 'color': (240, 200, 190)},
    {'elevation': 9000, 'color': (240, 200, 190)},
    {'elevation': 9001, 'color': (245, 220, 210)},
    {'elevation': 9500, 'color': (245, 220, 210)},
    {'elevation': 9501, 'color': (253, 235, 230)},
    {'elevation': 10000, 'color': (253, 235, 230)},
    {'elevation': 10001, 'color': (255, 255, 255)}]

In [ ]:
# cmap adapted from the original sierra map (elevations in feet)
norcal_cmap = [
    {'elevation': 0, 'color': (25.0, 125.0, 225.0)},
    {'elevation': 1, 'color': (110.0, 140.0, 100.0)},
    {'elevation': 100, 'color': (115.0, 150.0, 105.0)},
    {'elevation': 400, 'color': (140.0, 155.0, 115.0)},
    {'elevation': 500, 'color': (190.0, 204.0, 145.0)},
    {'elevation': 1000, 'color': (250.0, 250.0, 185.0)},
    {'elevation': 1500, 'color': (250.0, 205.0, 160.0)},
    {'elevation': 2000, 'color': (230.0, 180.0, 155.0)},
    {'elevation': 2500, 'color': (240.0, 200.0, 190.0)},
    {'elevation': 3000, 'color': (245.0, 220.0, 210.0)},
    {'elevation': 3500, 'color': (253.0, 235.0, 230.0)},
    {'elevation': 3600, 'color': (255.0, 255.0, 255.0)},
]

In [ ]:
# same as norcal_cmap but with less abrupt green-yellow transition at low elevations
norcal_cmap_2 = [
    {'elevation': 0, 'color': (25.0, 125.0, 225.0)},
    {'elevation': 1, 'color': (110.0, 140.0, 100.0)},
    {'elevation': 300, 'color': (115.0, 150.0, 105.0)},
    {'elevation': 600, 'color': (140.0, 155.0, 115.0)},
    {'elevation': 900, 'color': (190.0, 204.0, 145.0)},
    {'elevation': 1200, 'color': (250.0, 250.0, 185.0)},
    {'elevation': 1500, 'color': (250.0, 205.0, 160.0)},
    {'elevation': 1800, 'color': (230.0, 180.0, 155.0)},
    {'elevation': 2100, 'color': (240.0, 200.0, 190.0)},
    {'elevation': 2400, 'color': (245.0, 220.0, 210.0)},
    {'elevation': 2700, 'color': (253.0, 235.0, 230.0)},
    {'elevation': 3000, 'color': (255.0, 255.0, 255.0)},
]

In [ ]:
cmap_for_sf = [
    {'elevation': 0, 'color': (25.0, 125.0, 225.0)},
    {'elevation': 1, 'color': (110.0, 140.0, 100.0)},
    {'elevation': 100, 'color': (115.0, 150.0, 105.0)},
    {'elevation': 200, 'color': (140.0, 155.0, 115.0)},
    {'elevation': 300, 'color': (190.0, 204.0, 145.0)},
    {'elevation': 400, 'color': (250.0, 250.0, 185.0)},
    {'elevation': 500, 'color': (250.0, 205.0, 160.0)},
    {'elevation': 600, 'color': (230.0, 180.0, 155.0)},
    {'elevation': 700, 'color': (240.0, 200.0, 190.0)},
    {'elevation': 800, 'color': (245.0, 220.0, 210.0)},
    {'elevation': 900, 'color': (253.0, 235.0, 230.0)},
    {'elevation': 1000, 'color': (255.0, 255.0, 255.0)},
]

In [ ]:
cmap_for_moab = [
    {'elevation': 0, 'color': (25.0, 125.0, 225.0)},
    {'elevation': 4000, 'color': (110.0, 140.0, 100.0)},
    {'elevation': 4500, 'color': (115.0, 150.0, 105.0)},
    {'elevation': 5000, 'color': (140.0, 155.0, 115.0)},
    {'elevation': 5500, 'color': (190.0, 204.0, 145.0)},
    {'elevation': 6000, 'color': (250.0, 250.0, 185.0)},
    {'elevation': 6500, 'color': (250.0, 205.0, 160.0)},
    {'elevation': 7000, 'color': (230.0, 180.0, 155.0)},
    {'elevation': 7500, 'color': (240.0, 200.0, 190.0)},
    {'elevation': 8000, 'color': (245.0, 220.0, 210.0)},
    {'elevation': 9000, 'color': (253.0, 235.0, 230.0)},
    {'elevation': 10000, 'color': (255.0, 255.0, 255.0)},
]

In [ ]:
sns.palplot([np.array(row['color'])/255. for row in cmap_orig])

In [ ]:
sns.palplot(sns.color_palette('RdYlGn_r', 10))

In [ ]:
lblue, blue, lgreen, green, lred, red, lorange, orange, lpurple, purple, lbrown, brown = sns.color_palette('Paired')

In [ ]:
# paired cmap for moab (this is pretty ugly)
cmap_for_moab = [
    (4000, green),
    (4500, lgreen),
    (5000, orange),
    (5500, lorange),
    (6000, red),
    (6500, lred),
    (7000, blue),
    (7500, lblue),
    (8000, purple),
    (8500, lpurple),
    (11000, (1, 1, 1))
]

cmap_for_moab = [{'elevation': row[0], 'color': 255*np.array(row[1])} for row in cmap_for_moab]

In [ ]:
elevations = np.arange(4000, 7000, 250)
colors = sns.color_palette('RdYlGn', len(elevations))
cmap_for_moab = [{'elevation': e, 'color': np.array(c)*255} for e, c in zip(elevations, colors)]
cmap_for_moab.append({'elevation': 11000, 'color': (255, 255, 255)})

In [ ]:
def workflow(project_root, colormap, bounds=None, res=None, reset=False):
    '''
    Workflow for cropping/merging and processing NED13 ROIs
    '''

    # transform bounds from lat-lon to the NED13 CRS
    if bounds:
        d = managers.datasets.new_dataset('ned13', ned13_dirs[0], exists=True)
        bounds = managers.utils.transform(bounds, d.bandpath())

    proj = managers.DEMProject(
        project_root=project_root, 
        dataset_paths=ned13_dirs,
        bounds=bounds,
        res=res,
        reset=reset)
    
    # note that using get_operation('last') here assumes reset=True
    proj.warp(proj.get_operation('last'), crs='EPSG:3857', res=None)
    proj.hill_shade(proj.get_operation('last', 'warp'))
    proj.color_relief(proj.get_operation('last', 'warp'), colormap=colormap)
    proj.multiply_rgb([proj.get_operation('last', 'hill_shade'), proj.get_operation('last', 'color_relief')])
    proj.save_props()

In [ ]:
managers.utils.shell('gdalinfo %s' % managers.datasets.NED13Tile(ned13_dirs[0], exists=True).bandpath())

In [ ]:
# merge and crop all raw NED13 tiles
project_root='/home/keith/raster-projects/ned13-all/'
workflow(project_root, bounds=None, res=5e-3, reset=True, colormap=cmap_orig)

In [ ]:
# SF
project_root = '/home/keith/raster-projects/ned13-sf/'
workflow(project_root, cmap_for_sf, bounds=bounds['sf'], res=None, reset=True)

In [ ]:
# moab
project_root = '/home/keith/raster-projects/ned13-moab-paired-cmap/'
workflow(project_root, cmap_for_moab, bounds=bounds['moab'], res=0.0009, reset=True)

In [ ]:
# moab wide
project_root = '/home/keith/raster-projects/ned13-moab/'
workflow(project_root, cmap_for_moab, bounds=bounds['moab-wide'], res=0.0009, reset=True)

In [ ]:
# markleeville
project_root = '/home/keith/raster-projects/ned13-markleeville-low-res/'
workflow(project_root, cmap_markleeville, bounds=bounds['markleeville'], res=.0009, reset=True)

### Use a different colormap for an existing project

In [ ]:
proj = managers.DEMProject('/home/keith/raster-projects/ned13-adf-santa-cruz/')

In [ ]:
proj.save_props()

In [ ]:
# proj.color_relief(proj.get_operation('last', 'warp'), colormap=cmap_markleeville_banded)
proj.multiply_rgb([
    proj.get_operation('last', 'hill_shade'), 
    proj.get_operation('last', 'color_relief')])

### Rasterizing OSM roads to overlay (manually) on the final image

First, use pgsql2shp to dump the roads in the ROI to a shapefile.

In [ ]:
# for santa cruz
cmd = '''
pgsql2shp -f "/home/keith/raster-projects/ned13-adf-santa-cruz/roads.shp" 
-h localhost -u postgres osm_roads 
"select * from roads
 where ST_Intersects(ST_MakeEnvelope(-122.55, 36.929, -121.75, 37.65, 4326), geom)
 and code < 5120"
'''
cmd = cmd.replace('\n', '')

In [ ]:
# for markleeville
cmd = '''
pgsql2shp -f "/home/keith/raster-projects/ned13-markleeville/roads.shp" 
-h localhost -u postgres osm_roads 
"select * from roads
 where ST_Intersects(ST_MakeEnvelope(-120.194, 38.269, -119.361, 39.0, 4326), geom)
 and code < 5120"
'''
cmd = cmd.replace('\n', '')

In [ ]:
managers.utils.shell(cmd.replace('\n', ''))

Then, use rio CLI to rasterize and warp the shapefile. In the project directory, first convert from shapefile to geoJSON:

```fio dump roads.shp > roads.json```

Then rasterize the geoJSON and reproject:

```rio rasterize roads.json --default-value 255 --fill 0 --overwrite --res .00025 test00025.tif```

```rio warp test00025.tif test00025w.tif --like ned13-markleeville_multiply_20190601-104127.TIF ```

The resolution of .00025 is deliberately a bit greater than the native NED13 resolution, in order to make the roads more visible (if pixelated). 


### Debugging

In [ ]:
proj = managers.DEMProject(project_root='/home/keith/raster-projects/ned13-downsampled-norcal/')
with rasterio.open(proj.get_operation('last', 'hillshade').destination.path) as src:
    im_ds = src.read()

In [ ]:
proj = managers.DEMProject(project_root='/home/keith/raster-projects/ned13-berkeley/', reset=False)

In [ ]:
proj.texture_shade(proj.get_operation('last', 'warp'))

In [ ]:
proj.multiply([proj.get_operation('first', 'multiply'), proj.get_operation('last', 'texture_shade')], weight=.5)

In [ ]:
proj._serialize()